In [ ]:
# default_exp data.covariates

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#! pip install 'tokenizers==0.9.3'
#! pip install 'transformers==3.5.0'

# Covariate Analysis and Feature Engineering 
>
>@danaderp 11.17.20 .
>

In [ ]:
#export
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import time

In [ ]:
#export
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
#export
import functools 
from operator import or_
from collections import Counter
import datetime

In [ ]:
#export
import sentencepiece as spm
import pickle
from pathlib import Path
from tokenizers import Tokenizer

In [ ]:
#export
from pandas.plotting import scatter_matrix
from pandas.plotting import bootstrap_plot
from sklearn import preprocessing

In [ ]:
#export
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Fixing parameters

In [ ]:
#nbs_experiments/results/analyses/rnn_layers1_vocab10000_embed256_units1024
#df_buggy.to_json(path / "buggy.jsonl", orient="records", lines=True)
dvc_path = Path('../dvc-icodegen/')
def params():
    return {
        'tokenizer':dvc_path / 'models/bpe/tokenizer-java-v1.json',
        'tb_01':dvc_path / 'nbs_experiments/results/analyses/rnn_layers1_vocab10000_embed256_units1024/bug_fix_error_taxonomy.jsonl',
        'tb_02':dvc_path / 'nbs_experiments/results/analyses/rnn_layers1_vocab10000_embed256_units1024/bug_fix_cross_entropy.jsonl'
    }

In [ ]:
params = params()

In [ ]:
#tst
logging.info( params['tokenizer'] )

In [18]:
df_error_tax = pd.read_json(
            params['tb_01'], orient="records", lines=True)

2021-06-23 20:32:32,334 : INFO : Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2021-06-23 20:32:32,335 : INFO : Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-06-23 20:32:32,335 : INFO : NumExpr defaulting to 8 threads.


# First Covariate Analysis: Token Size

In [ ]:
#Upload tokenizers here
#Use Tokenizer recent version
#Load tokenizer
#Update files (for one time) the main json files to add the column of tokenizers
#Count the tokens per method
#Visualize the most popular tokens (and less popular)

In [67]:
class TokenizeHF():
    def __init__( self ):
        """
        :param method: Code snippet (plain text)
        :returns: Encoded result using the provided tokenizer
        """
        #Load tokenizer
        self.tokenizer = Tokenizer.from_file( str( params['tokenizer'] ) )
        pass
    
    #@staticmethod
    def encodingHF(self, method ):
        output = {}
        # encode method and then convert to format that hf models expect
        encoding = self.tokenizer.encode("<sos>" + method)
        output["input_ids"] = tf.expand_dims(
             tf.convert_to_tensor(encoding.ids, dtype=tf.int32), 0
        )
        output["attention_mask"] = tf.expand_dims(
             tf.convert_to_tensor(encoding.attention_mask, dtype=tf.int32), 0
        )
        return encoding
    
    #@staticmethod
    def decodingHF( self, idss ):
        return self.tokenizer.decode( idss , skip_special_tokens=False )
        #return self.tokenizer.convert_ids_to_tokens(idss )

In [37]:
code = df_error_tax['code']

In [38]:
code[0]

'<private> <void> success<(>io.netty.channel.Channel channel<)> <{><n>    org.mycat.netty.mysql.MySQLHandshakeHandler.logger.debug<(>"success info <return> <for>m MySQLHandshakeHandler"<)><;><n>    io.netty.buffer.ByteBuf out <=> channel.alloc<(><)>.buffer<(><)><;><n>    org.mycat.netty.mysql.OK ok <=> <new> org.mycat.netty.mysql.OK<(><)><;><n>    ok.sequenceId <=> 2<;><n>    ok.setStatusFlag<(>Flags.SERVER_STATUS_AUTOCOMMIT<)><;><n>    out.writeBytes<(>ok.toPacket<(><)><)><;><n>    channel.writeAndFlush<(>out<)><;><n><}>'

In [68]:
class_tokenize = TokenizeHF() 
input_ids = class_tokenize.encodingHF( method = code[0] )
#input_ids = input_ids['input_ids']
logging.info(input_ids)
logging.info(input_ids.ids)
logging.info(input_ids.tokens)

2021-06-23 21:15:35,433 : INFO : Encoding(num_tokens=112, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
2021-06-23 21:15:35,434 : INFO : [1, 35, 189, 50, 3905, 83, 1702, 105, 3342, 487, 105, 2867, 105, 1821, 2647, 84, 189, 79, 91, 195, 2841, 105, 4047, 2853, 105, 3342, 487, 105, 168, 2387, 1366, 105, 136, 180, 2298, 3943, 5731, 1072, 105, 1926, 105, 1025, 83, 93, 6150, 2399, 189, 38, 189, 23, 168, 464, 180, 2298, 3943, 5731, 1072, 93, 84, 85, 91, 195, 3644, 105, 3342, 487, 105, 2057, 105, 881, 3453, 1062, 189, 53, 2647, 105, 212, 2586, 83, 84, 105, 2057, 83, 84, 85, 91, 195, 2841, 105, 4047, 2853, 105, 3342, 487, 105, 168, 2387, 1366, 105, 4236, 4959, 189, 53, 189, 33, 2841, 105, 4047, 2853, 105, 3342, 487]
2021-06-23 21:15:35,434 : INFO : ['<sos>', '<private>', 'Ġ', '<void>', 'Ġsuccess', '<(>', 'io', '.', 'net', 'ty', '.', 'channel', '.', 'Channel', 'Ġchannel', '<)>', 'Ġ', '<{>', '<n>', 'ĠĠĠ', 'Ġorg', '.', 'my', 'cat', '.', 'net', 'ty',

In [69]:
class_tokenize.decodingHF( input_ids.ids )

'<sos><private> <void> success<(>io.netty.channel.Channel channel<)> <{><n>    org.mycat.netty.mysql.MySQLHandshakeHandler.logger.debug<(>"success info <return> <for>m MySQLHandshakeHandler"<)><;><n>    io.netty.buffer.ByteBuf out <=> channel.alloc<(><)>.buffer<(><)><;><n>    org.mycat.netty.mysql.OK ok <=> <new> org.mycat.netty'

# Second Covariate Analysis: SE Metrics
> This module provides a tool for computing metrics (from static analysis) for python source code using Using <a href="https://github.com/mauricioaniche/ck">CK Package</a>

CK is a java package (jar) which is going to be executed from terminal. It requires the code which is going to be analyzed to be located at <i>physical</i> files. For that reason, the dataset is going to be used to produce some <i>.java</i> files.

Each record, corresponds to a individual class. When working with method-level snippets, "articial" classes are created for performing the analysis.

In [ ]:
# export
def write_dataset_to_files(df_series, destination_path):
    """
    Function to generate .java files.
    
    Params:
    # df_series: Pandas Series (DataFrame column) with the source code records.
    # destination_path: (str) Absolute path to be used as directory for the generated files.
    
    Returns:
    
    Collection of paths for the corresponding java files.
    
    """
    java_template = 'public class <class_name>{\n    <code_snippet>\n}'
    
    if not os.path.exists(destination_path):
        logging.info('Creating directory.')
        os.mkdir(destination_path)
    
    logging.info("Generating physical .java files.")
    
    file_paths = []
    for idx, value in df_series.iteritems():
        class_name = f'ClassRecord{idx}'
        code = java_template.replace('<class_name>', class_name)
        code = code.replace('<code_snippet>', value)
        file_path = f'{destination_path}/{class_name}.java'
        with open(file_path, 'w') as file:
            file.write(code)
            file_paths.append(file_path)
            
    return file_paths

In [ ]:
# export

def jarWrapper(*args):
    process = Popen(['java', '-jar']+list(args), stdout=PIPE, stderr=PIPE)
    ret = []
    while process.poll() is None:
        line = process.stdout.readline()
        if line != '' and line.endswith(b'\n'):
            ret.append(line[:-1])
    stdout, stderr = process.communicate()
    
    ret += stdout.split(b'\n')
    if stderr != '':
        ret += stderr.split(b'\n')
        
    if '' in ret:
        ret.remove('')
    return ret

In [ ]:
# export

class JavaAnalyzer():
    """
    Class get metrics f
    """
    def __init__(self, ck_jar_path):
        self.ck_jar_path = ck_jar_path
    
    def compute_metrics(self, df_series, files_destination_path):
        """
        Computes metrics for a pandas series of java source code snippets
        
        Params
        # df_series: Pandas series (df column) containing java source snippets
        # files_destination_path: Path indicating where the physical .java files are going to be created (for metrics computation)
        
        Returns:
        
        Pandas Dataframe containing metrics
        
        """
        file_paths = write_dataset_to_files(df_series, files_destination_path)
        self.__call_ck_package(files_destination_path)
        metrics_df = self.__get_metrics_df()
        self.__remove_csv_files()
        self.__remove_tmp_java_files(file_paths)
        
        return metrics_df
        
    def __call_ck_package(self, files_path):
        """
        Performs call to external .jar package.
        """
        args = [self.ck_jar_path, files_path, 'false', '0', 'True']
        result = jarWrapper(*args)
        logging.info(f'CK package produced this output:\n{result}')
        
    def __get_metrics_df(self):
        """
        Reads report files (csv) generated by the CK package.
        
        Returns:
        
        Pandas Dataframe containing appropriate metrics
        """
        class_metrics_df = pd.read_csv('class.csv')
        # method_metrics_df = pd.read_csv('method.csv')

        # merged_df = pd.merge(left = class_metrics_df, right = method_metrics_df, left_on='file', right_on='file')

        appropriate_columns = ['file','class', 'wmc', 'totalMethodsQty', 'staticMethodsQty', 'publicMethodsQty', 'privateMethodsQty',
                          'protectedMethodsQty', 'defaultMethodsQty', 'abstractMethodsQty', 'finalMethodsQty','synchronizedMethodsQty',
                          'totalFieldsQty', 'staticFieldsQty', 'publicFieldsQty', 'privateFieldsQty', 'protectedFieldsQty',
                          'defaultFieldsQty', 'visibleFieldsQty', 'finalFieldsQty', 'synchronizedFieldsQty',
                          'nosi', 'loc', 'returnQty', 'loopQty', 'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty',
                          'stringLiteralsQty', 'numbersQty', 'assignmentsQty', 'mathOperationsQty', 'variablesQty', 'maxNestedBlocksQty',
                          'anonymousClassesQty', 'innerClassesQty', 'lambdasQty', 'uniqueWordsQty', 'modifiers']

        class_metrics_df = class_metrics_df[appropriate_columns]

        return class_metrics_df
    
    def __remove_csv_files(self):
        """
        Removes files generated by CK package.
        """
        if os.path.exists('class.csv'):
            os.remove('class.csv')
        if os.path.exists('method.csv'):
            os.remove('method.csv')
        if os.path.exists('field.csv'):
            os.remove('field.csv')
            
    def __remove_tmp_java_files(self, paths):
        """
        Removes the temporary generated java files.
        """
        for file_path in paths:
            os.remove(file_path)

In [ ]:
# General parameters for testing

def get_default_params():
    return {
    'ck_jar_path': 'ck_metrics_tool/ck-metrics.jar',
    'search_net_ds_path': '/tf/main/dvc-ds4se/code/searchnet/clean_java.csv',
    'sampling_size': 100,
    'physical_files_path': '/tf/main/nbs/test_data/test_metrics'
}

# Data Transformations 

In [ ]:
#Avg Number of Subwords 
count_subwords = [len( eval(mtd) ) for mtd in df_train['code_tokens'].values]

In [ ]:
df_train['count_tokens'] = count_subwords

In [ ]:
#BPE 

# Descriptive Statistics

All Java Set

In [ ]:
java_df.describe()

,code_len,bpe32_len
count,467203.000000,467203.000000
mean,113.079653,146.274557
std,189.121245,303.804009
min,20.000000,20.000000
25%,42.000000,50.000000
50%,67.000000,81.000000
75%,122.000000,150.000000
max,27192.000000,52975.000000


All Java Partition

In [ ]:
df_train.describe()

,code_len,bpe32_len
count,384868.000000,384868.000000
mean,113.720826,147.151002
std,194.007951,313.904001
min,20.000000,20.000000
25%,42.000000,50.000000
50%,67.000000,81.000000
75%,122.000000,151.000000
max,27192.000000,52975.000000


In [ ]:
logging.info([ p.describe() for p in list_all_partitions])

2020-11-17 12:31:54,629 : INFO : [            code_len      bpe32_len
count  384868.000000  384868.000000
mean      113.720826     147.151002
std       194.007951     313.904001
min        20.000000      20.000000
25%        42.000000      50.000000
50%        67.000000      81.000000
75%       122.000000     151.000000
max     27192.000000   52975.000000,            code_len     bpe32_len
count  14605.000000  14605.000000
mean      94.331736    120.097843
std      115.802231    171.079255
min       21.000000     21.000000
25%       39.000000     46.000000
50%       59.000000     71.000000
75%      104.000000    127.000000
max     3099.000000   5747.000000,            code_len     bpe32_len
count  25011.000000  25011.000000
mean     114.274599    148.204710
std      166.432695    245.938732
min       21.000000     22.000000
25%       43.000000     52.000000
50%       69.000000     84.000000
75%      125.000000    155.000000
max     5685.000000  10015.000000,            code_len     bpe

In [ ]:
logging.info([ stats.median_absolute_deviation(p['code_len'].values) for p in list_all_partitions])

2020-11-17 12:29:30,408 : INFO : [45.9606, 37.065, 47.4432, 44.477999999999994]


Train Partition

In [ ]:
# Frequent Characters
train_tokens = df_train.code_tokens.values

In [ ]:
train_vocab_tokens = [ eval(method)  for method in train_tokens ] #Evaluating given tokens

In [ ]:
train_counter_tokens = [ Counter(method) for method in train_vocab_tokens ] #Counting the words

In [ ]:
train_counter = functools.reduce(lambda a,b : a+b, train_counter_tokens ) ## [Warning! Time Consuming]

In [ ]:
train_counter

NameError: name 'train_counter' is not defined

In [ ]:
#Persisting the counter object
with open(params['eda']+'['+ str(datetime.datetime.now()) +']-codesearchnet_token_counts.pickle', 'wb') as outputfile:
    pickle.dump( train_counter, outputfile )